In [1]:
import pandas as pd
import datetime
import math
import numpy as np

# 1. Set up credentials

In [4]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = './ga-project-318200-0eacacd12786.json'
VIEW_ID = '208505317'

In [2]:
def BuildCredentials(KEY_FILE_LOCATION,SCOPES):
    
    from apiclient.discovery import build
    from oauth2client.service_account import ServiceAccountCredentials
    
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    analytics = build('analyticsreporting', 'v4', credentials=credentials)
    
    return analytics

In [5]:
cred = BuildCredentials(KEY_FILE_LOCATION,SCOPES)

In [29]:
def getTotalVisitor(cred,pageTokenVar):
    req = cred.reports().batchGet(
        body = {
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '2021-04-01', 'endDate': '2021-06-27'}],
          'metrics': [{'expression': 'ga:users'}],
          'metricFilterClauses':[{
              'filters':[{
                  'metricName':'ga:users',
                  'operator':'GREATER_THAN',
                  'comparisonValue':'2'
              }]
              
          }],
          'dimensions': [{'name': 'ga:pagePath'},
                         {'name': 'ga:date'}],
          'dimensionFilterClauses':[{
              'filters' :[{
                  'dimensionName':'ga:pagePath',
                  'operator':'IN_LIST',
                  'expressions':['/measurement-scale/','/']
              }]
          }],
          'pageSize': 10000,
          'pageToken': pageTokenVar,
          'samplingLevel': 'LARGE'
        }]
      }
    
    ).execute()
    
    return req

In [30]:
df = getTotalVisitor(cred,'0')

In [31]:
dimensi_names = df['reports'][0]['columnHeader']['dimensions']
metrics_names = df['reports'][0]['columnHeader']['metricHeader']['metricHeaderEntries'][0]['name']

dataraw = df['reports'][0]['data']['rows']

In [32]:
ls_total_visitor = []
for k in range(len(dataraw)):
    page = dataraw[k]['dimensions'][0]
    str_date = dataraw[k]['dimensions'][1]
    date = datetime.date(int(str_date[0:4]),int(str_date[4:6]),int(str_date[6:8]))
    total_user = int(dataraw[k]['metrics'][0]['values'][0])
    ls_total_visitor.append([page,date,total_user])

In [33]:
df_visitor = pd.DataFrame(ls_total_visitor)
df_visitor.columns = ['page','date','total_user']

In [34]:
df_visitor

,page,date,total_user
0,/,2021-04-04,3
1,/,2021-04-23,3
2,/,2021-04-28,3
3,/measurement-scale/,2021-04-06,3
4,/measurement-scale/,2021-04-07,3
5,/measurement-scale/,2021-04-08,4
6,/measurement-scale/,2021-04-15,6
7,/measurement-scale/,2021-04-16,5
8,/measurement-scale/,2021-04-19,7
9,/measurement-scale/,2021-04-23,3


# Refference
1. https://developers.google.com/analytics/devguides/reporting/core/v4/basics
2. https://developers.google.com/analytics/devguides/reporting/core/v4/rest/v4/reports/batchGet#dynamicsegment
3. https://www.youtube.com/watch?v=Xpx--7g6FKk&ab_channel=TobiasWillmann
4. https://www.jcchouinard.com/google-analytics-api-using-python/